# RSA Mega Gift Pak

In [91]:
from gmpy2 import invert, powmod, iroot, next_prime, is_prime
from functools import reduce
from Crypto.Util.number import long_to_bytes
import binascii
from math import gcd

In [7]:
frames = ["Frame/Frame" + str(i) for i in range(21)]
print(frames)

['Frame/Frame0', 'Frame/Frame1', 'Frame/Frame2', 'Frame/Frame3', 'Frame/Frame4', 'Frame/Frame5', 'Frame/Frame6', 'Frame/Frame7', 'Frame/Frame8', 'Frame/Frame9', 'Frame/Frame10', 'Frame/Frame11', 'Frame/Frame12', 'Frame/Frame13', 'Frame/Frame14', 'Frame/Frame15', 'Frame/Frame16', 'Frame/Frame17', 'Frame/Frame18', 'Frame/Frame19', 'Frame/Frame20']


In [88]:
def extract_frame(frame):
    with open(frame, 'r') as f:
        data = f.read()
    n , e , c = int(data[:256] , 16) , int(data[256:512] , 16) , int(data[512:] , 16)
    return n, e, c

def dec(c, d, n):
    return long_to_bytes(pow(c, d, n)).split(b'\x00')[-1].decode()

def iter_all(func, myList):
    res = []
    for i in range(len(myList)):
        for j in range(i+1, len(myList)):
            ret = func(myList[i], myList[j])
            if ret:
                res.append([i, j, ret])
    return res

### Same Module Attack

In [49]:
def same_module_attack(N , e1 , e2 , c1 , c2):
    d1 = invert(e1 , e2)
    d2 = (d1 * e1 - 1) // e2
    true_c2 = invert(c2 , N)
    return (powmod(c1 , d1 , N) * powmod(true_c2 , d2 , N)) % N

def same_module_attack_for_frame(frame1, frame2):
    n1 , e1 , c1 = extract_frame(frame1)
    n2 , e2 , c2 = extract_frame(frame2)
    if n1 == n2:
        m = same_module_attack(n0,e0,e4,c0,c4)
        return binascii.a2b_hex(hex(m)[2:])[-8:].decode()
    return None

In [119]:
iter_all(same_module_attack_for_frame, frames)

[[0, 4, 'My secre']]

### Low e Attack

In [54]:
def CRT(mi, ai):
    M = reduce(lambda x, y: x * y, mi)
    ai_ti_Mi = [a * (M // m) * invert(M // m, m) for (m, a) in zip(mi, ai)]
    return reduce(lambda x, y: x + y, ai_ti_Mi) % M

def low_e_attack(n, e, c):
    m = CRT(n, c)
    tmp = iroot(m, e)
    if tmp[1] == 1:
        return tmp[0]
    return 0

In [55]:
n = []
c = []
index = [3, 8, 12, 16, 20]
for i in index:
    nt, _, ct = extract_frame(frames[i])
    n.append(nt)
    c.append(ct)
m = low_e_attack(n, 5, c)
m = long_to_bytes(m).split(b'\x00')[-1].decode()
print(m)

t is a f


### Factor Collision

In [73]:
def factor_collision_attack(frame1, frame2):
    n1, e1, c1 = extract_frame(frame1)
    n2, e2, c2 = extract_frame(frame2)
    p = gcd(n1,n2)
    if p == 1:
        return None
    q1 = n1 // p
    q2 = n2 // p
    try:
        d1 = invert(e1, (p-1)*(q1-1))
        d2 = invert(e2, (p-1)*(q2-1))
    except:
        return None
    return dec(c1 ,d1, n1), dec(c2, d2, n2)

In [89]:
iter_all(factor_collision_attack, frames)

[[1, 18, ('. Imagin', 'm A to B')]]

### Fermat

In [83]:
def fermat_dec_attack(frame):
    n, e, c = extract_frame(frame)
    p_q = iroot(n, 2)[0]
    for _ in range(20000):
        p_q += 1
        if iroot(p_q**2 - n,2)[1] == 1:
            tmp = iroot(p_q**2 - n,2)[0]
            p = (p_q + tmp)
            q = (p_q - tmp)
            phi = (p-1)*(q-1)
            d = invert(e, phi)
            return dec(c, d, n)
    return None

In [90]:
for i in range(len(frames)):
    m = fermat_dec_attack(frames[i])
    if m:
        print(i, m)

10 will get


### Pollard p-1

In [114]:
def Pollard_p_1(N):
    a = 2
    while a < N-2:
        f = a
        for n in range(1, 200000):
            f = powmod(f, n, N)
            if n % 2 == 0:
                d = gcd(f-1, N)
                if 1 < d < N:
                    return d
        print(a)
        a += 1

In [117]:
index = [2, 6, 19]
for i in index:
    n, e, c = extract_frame(frames[i])
    p = Pollard_p_1(n)
    q = n // p
    phi = (p-1) * (q-1)
    d = invert(e,phi)
    m = dec(c, d, n)
    print(i, m)

2  That is
6  "Logic 
19 instein.
